In [ ]:
try:
    # se estiver no colab vai rodar o bloco abaixo
    from google.colab import data_table, drive
    drive.mount('/content/drive', force_remount=True)

    data_table.enable_dataframe_formatter()
    data_table.DataTable.max_columns = 50
    path = "/content/drive/MyDrive/doutorado/videos_utah"  # caminho do google drive
except:
    # se nao estiver no colab vai rodar o bloco abaixo
    path = "."  # caminho local

In [ ]:
import numpy as np
import cv2
import os
import glob
from google.colab.patches import cv2_imshow
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.colors import TwoSlopeNorm

%matplotlib inline
%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}

plt.rcParams.update({'font.size': 22})

clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))

kernel2 = np.array([[0, -1, 0],
                    [-1, 5, -1],
                    [0, -1, 0]])

m2km = lambda x, _: f'{x/1000:g}'

In [ ]:
def video_stacker(dname, thre):
    global path
    flist = glob.glob('%s/%s_bmp/Y*H*.bmp' % (path, dname))
    lma = pd.read_csv('%s/csv/%s.csv' % (path, dname))
    print(lma.head())

    frame_orig = cv2.imread(flist[0], cv2.IMREAD_COLOR)
    h, w, c = frame_orig.shape

    frame_stack = np.zeros(shape=[h, w, 3], dtype=np.uint8)
    frame_lma = np.zeros(shape=[h, w, 3], dtype=np.uint8)
    frame_dual = np.zeros(shape=[3*h, 2*w, 3], dtype=np.uint8)

    os.system("rm '%s/%s.*'" % (path, dname))

    video_stack = cv2.VideoWriter('%s/%s.stack.mp4' % (path, dname),
                                  cv2.VideoWriter_fourcc(*'mp4v'),
                                  60,
                                  (w, h)
                                  )

    video_orig = cv2.VideoWriter('%s/%s.original.mp4' % (path, dname),
                                 cv2.VideoWriter_fourcc(*'mp4v'),
                                 60,
                                 (w, h)
                                 )

    video_lma = cv2.VideoWriter('%s/%s.lma.mp4' % (path, dname),
                                cv2.VideoWriter_fourcc(*'mp4v'),
                                60,
                                (w, h)
                                )

    video_dual = cv2.VideoWriter('%s/%s.dual.mp4' % (path, dname),
                                 cv2.VideoWriter_fourcc(*'mp4v'),
                                 60,
                                 (2*w, 3*h)
                                 )

    px = 1/plt.rcParams['figure.dpi']  # pixel in inches
    fig = plt.figure(figsize=(2*w*px, 2*h*px))

    ax = fig.add_subplot(111, label="2", facecolor='none')
    fig.subplots_adjust(left=0.1, right=0.98, top=0.92, bottom=0.15)

    ax.yaxis.set_major_formatter(m2km)
    ax.set(xlabel="Lat (°)", ylabel="Alt (km)", xlim=(lma.lat.min(), lma.lat.max()), ylim=(0, lma.alt.max()))
    txt = fig.text(0.5, 0.95, f"""teste""", horizontalalignment='center', family="monospace", size="smaller")

    p = ax.scatter(lma.lat[0], lma.alt[0], c=lma["dt"][0], cmap="jet", norm=TwoSlopeNorm(0), s=60, alpha=0.8, marker=".")

    tf = i = 0
    for f in sorted(flist):
        if not i % 100:
            print(i, len(flist), f)

        i += 1
        frame_orig = cv2.imread(f, cv2.IMREAD_COLOR)
        frame_orig = cv2.detailEnhance(frame_orig, sigma_s=10, sigma_r=0.15)

        thresh = frame_orig > 220
        thresh = thresh.sum()

        if thresh > thre:
            print(thresh, end=" ")
        #     break

        frame_stack = np.maximum(frame_stack, frame_orig)

        tf = os.path.splitext(os.path.basename(f))[0][0:26]
        tf = pd.to_datetime(tf, format='Y%Y%m%dH%H%M%S.%f')
        txt.set_text("%s %0.1fms" % (tf.strftime('%Y-%m-%d %H:%M:%S'), float(tf.strftime('%f'))/1000))
        tf = (tf - tf.replace(hour=0, minute=0, second=0, microsecond=0)).total_seconds()
        lmax = lma[(lma['time'] <= tf)]

        p.set_offsets(lmax[["lat", "alt"]])
        p.set_array(lmax["dt"])

        fig.canvas.draw()  # redraw the canvas
        frame_lma = np.array(fig.canvas.renderer.buffer_rgba())  # convert canvas to image
        frame_lma = cv2.cvtColor(frame_lma, cv2.COLOR_RGBA2BGR)  # frame_lma is rgba, convert to opencv's default bgr

        frame_dual = np.concatenate((frame_orig, frame_stack), axis=1)
        frame_dual = np.concatenate((frame_dual, frame_lma), axis=0)

        video_orig.write(frame_orig)
        video_stack.write(frame_stack)
        video_dual.write(frame_dual)
        video_lma.write(frame_lma)

    cv2.imwrite('%s/%s.stack.jpg' % (path, dname), frame_stack)
    cv2.imwrite('%s/%s.dual.jpg' % (path, dname), frame_dual)
    cv2.imwrite('%s/%s.lma.jpg' % (path, dname), frame_lma)
    cv2_imshow(frame_dual)

    video_orig.release()
    video_stack.release()
    video_dual.release()
    video_lma.release()


video_stacker("20230718H202127", 200)
video_stacker("20230817H203603", 16000)